In [29]:
import ROOT
import root_numpy as rnpy
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
import pickle
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM

In [43]:
# build here the keras model
def RNN_classifier():
    model = Sequential()
    
    model.add(LSTM(4, input_shape = (6, 8)))
    
    # make an output layer with just 1 output -> for a binary classification problem: b-jet / not b-jet
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  
    return model

In [11]:
batch_size_jets = 25
batch_size_tracks = 25
read_pos_jets = 0
read_pos_tracks = 0
number_chunks = 0
chunks_limit = 1

In [12]:
while number_chunks < chunks_limit:
    number_chunks += 1
    
    # read in new chunk of jet and track data
    d1 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
                                 treename = "tagVars/ttree", start = read_pos_jets, stop = read_pos_jets + batch_size_jets))
    d2 = pd.DataFrame(rnpy.root2array("/mnt/t3nfs01/data01/shome/jpata/btv/gc/TagVarExtractor/GCa08e5e237323/TT_TuneCUETP8M1_13TeV-powheg-pythia8/job_0_out.root",
                                 treename = "tagVars/ttree_track", start = read_pos_tracks, stop = read_pos_tracks + batch_size_tracks))
    
    # figure out where the next chunk should start so that we don't count any jets multiple times
    last_tracks = (int)(d2.tail(1)['Track_jetIndex'].iloc[0]-1)
    last_jet = (int)(d1.tail(1)['Jet_jetIndex'].iloc[0]-1)
    read_pos_jets += (d1.loc[d1['Jet_jetIndex'] == last_tracks].index[-1] + 1)
    read_pos_tracks += (d2.loc[d2['Track_jetIndex'] == last_tracks].index[-1] + 1)

    # add the track data to the jet list
    d1['track_data'] = pd.np.empty((len(d1.index),0)).tolist()
    
    # iterate over the track list to join jets with the tracks belonging to them
    for irow, row in d2.iterrows():
        # these are the track data of the current track:
        tracks = row[["Track_pt", "Track_eta", "Track_phi", "Track_dxy", "Track_dz", "Track_IP", "Track_IP2D", "Track_length"]].as_matrix()
        jet_index = int(row["Track_jetIndex"])
        if jet_index > last_tracks:
            break
        table_index = d1.loc[d1['Jet_jetIndex'] == jet_index].index[0]

        # append the tracks data to the matching jet in the main table
        d1['track_data'][table_index].append(tracks)
    
    # now divide the jets and put them in separate lists, according to their flavour
    jets_b = []
    jets_l = []
    jets_c = []

    # iterate over the jet list, with already matched tracks
    for irow, row in d1.iterrows():
        jet_index = int(row["Jet_jetIndex"])
        if jet_index > last_tracks:
            break

        flavour = int(row["Jet_flavour"])

        # select the right list this jet belongs to
        if abs(flavour) == 5:
            jets = jets_b
        elif abs(flavour) == 4:
            jets = jets_c
        else:
            jets = jets_l

        # add the new jet to the list
        jets += [(row["Jet_pt"], row["Jet_eta"], row["Jet_phi"], row["Jet_mass"], flavour, row["track_data"])]
        
    # now, have sorted jets in three lists, can use them directly for training!

In [16]:
test = jets_b[0][-1]

In [37]:
# sort the list of tracks in reverse pt-order: i.e. put the hardest track first!
x=sorted(test, key = lambda tracks: tracks[0], reverse = True)
y=1

In [44]:
model = RNN_classifier()

In [50]:
model.fit(np.array(np.array(x)),y)

Exception: Error when checking model input: expected lstm_input_3 to have 3 dimensions, but got array with shape (6, 8)

In [49]:
np.array(np.array(x))

array([[  1.20859375e+01,   1.08896148e+00,  -1.66410536e-01,
          2.64635198e-02,   1.30043542e-02,   2.76068579e-02,
          2.64635142e-02,   3.06698036e+00],
       [  1.09843750e+01,   1.06442451e+00,  -1.87506616e-01,
          3.31831202e-02,   3.77941430e-02,   4.05466929e-02,
          3.31831314e-02,   1.70997262e+00],
       [  7.34765625e+00,   1.11990726e+00,  -4.97098789e-02,
         -2.66015623e-02,   1.53442379e-03,   2.66169552e-02,
          2.66015567e-02,   3.82213205e-01],
       [  2.78320312e+00,   8.39381099e-01,  -3.69839042e-01,
         -2.19482416e-03,  -2.47070310e-03,  -2.83815456e-03,
         -2.19482300e-03,   1.27673065e-02],
       [  2.53906250e+00,   1.15927613e+00,  -6.29940480e-02,
         -1.54980468e-02,  -4.01953124e-02,   2.77004987e-02,
          1.54980412e-02,   3.76480132e-01],
       [  2.21289062e+00,   1.11386454e+00,  -1.45703152e-01,
          1.98339857e-02,  -4.96337889e-03,  -2.00509205e-02,
         -1.98339894e-02,   1.6

In [ ]:
6